# Apriori Association

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from mlxtend import frequent_patterns

%matplotlib inline

In [65]:
df = pd.read_csv("Transactions.csv")
df

,TID,List
0,T100,"M, O, N, K, E, Y"
1,T200,"D, O, N, K, E, Y"
2,T300,"M, A, K, E"
3,T400,"M, U, C, Y"
4,T500,"C, O, K, I, E"


In [3]:
df.head()

df = df.drop(['TID'], axis=1)
print df

               List
0  M, O, N, K, E, Y
1  D, O, N, K, E, Y
2        M, A, K, E
3        M, U, C, Y
4     C, O, K, I, E


## With code

In [6]:
def convertToOnehot(df, col):
#     convert a single column like list here into separate columns 
    return df.join(df[col].str.get_dummies(", ")).drop(["List"], axis=1)

In [7]:
def combineCols(df, col1, col2):
    return (df[col1] & df[col2])

In [8]:
def checkForCombinations(combList, combTuple, r):
    isPresent = True
    for comb in combinations(combTuple, r):
        if comb not in combList:
            isPresent = False
            break
    return isPresent

In [9]:
def getCount(df, col):
    return len(df.loc[df[col] == 1])

In [10]:
df.sum()

List    M, O, N, K, E, YD, O, N, K, E, YM, A, K, EM, U...
dtype: object

In [11]:
df

,List
0,"M, O, N, K, E, Y"
1,"D, O, N, K, E, Y"
2,"M, A, K, E"
3,"M, U, C, Y"
4,"C, O, K, I, E"


def apriori(df, col, s_count):
    #not required
    #df = pd.DataFrame(df[col])
    
    # items hold - column list
    items = df.columns
    # countdf = count support 
    countDf = df.sum()
    # filter min support and store it as dictionary
    countDict = countDf.loc[countDf >= s_count].to_dict()
    print(countDict)
    # items contains all keys
    items = list(countDict.keys())
    combHist = []
    # combPrev contain previous combinations 
    combPrev = list(combinations(items, 1))
    combNext = []
    

In [58]:
def apriori(df, col, s_count):
    df = pd.DataFrame(df[col])
    df = convertToOnehot(df, col)
    items = df.columns
    countDf = df.sum()
    countDict = countDf.loc[countDf >= s_count].to_dict()
    print(countDict)
    items = list(countDict.keys())
    combHist = []
    combPrev = list(combinations(items, 1))
    combNext = []
    for i in range(2, len(items) - 1):
        for comb in combinations(items, i):
            if checkForCombinations(combPrev, comb, (i-1)):
                combProp = "_".join(str(c) for c in comb)
                col1 = "_".join(str(c) for c in comb[:-1])
                col2 = str(comb[-1])
                df[combProp] = combineCols(df, col1, col2)
                countTemp = getCount(df, combProp)
                if countTemp >= s_count:
                    combNext.append(comb)
                    countDict[combProp] = getCount(df, combProp)
                    #print(countDict)
        combHist.append(combPrev)
        combPrev = combNext
        print(combPrev)
    return countDict

In [89]:
countDict = apriori(df, 'List', 3)

{'Y': 3L, 'M': 3L, 'K': 4L, 'E': 4L, 'O': 3L}
[('K', 'E'), ('K', 'O'), ('E', 'O')]
[('K', 'E'), ('K', 'O'), ('E', 'O'), ('K', 'E', 'O')]


## Using Built in function 

In [90]:
oneHotDf = convertToOnehot(df, 'List')
cols = ['M', 'O', 'N', 'K', 'E','Y','A','C','D','I','U']
oneHotDf[cols] = oneHotDf[cols].astype('bool')

In [91]:
oneHotDf = oneHotDf.drop(['TID'], axis=1)
oneHotDf

,A,C,D,E,I,K,M,N,O,U,Y
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,False,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [92]:
freq_itemsets = frequent_patterns.apriori(oneHotDf, min_support=0.20, use_colnames=True)

In [103]:
minConfidence = 0.8
rules = frequent_patterns.association_rules(freq_itemsets, metric="confidence", min_threshold=1)
rules[rules.support > 0.5]

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(O),(E),0.6,0.8,0.6,1.0,1.25,0.12,inf
172,(K),(E),0.8,0.8,0.8,1.0,1.25,0.16,inf
173,(E),(K),0.8,0.8,0.8,1.0,1.25,0.16,inf
241,"(K, O)",(E),0.6,0.8,0.6,1.0,1.25,0.12,inf
242,"(E, O)",(K),0.6,0.8,0.6,1.0,1.25,0.12,inf
243,(O),"(K, E)",0.6,0.8,0.6,1.0,1.25,0.12,inf
552,(O),(K),0.6,0.8,0.6,1.0,1.25,0.12,inf
